# Data Extraction  

- The actual scorecard for the match below can be viewed at [Cricinfo webpage.](https://www.espncricinfo.com/series/8048/scorecard/335982/royal-challengers-bangalore-vs-kolkata-knight-riders-1st-match-indian-premier-league-2007-08)

- The ball-by-ball data can be downloaded from (https://cricsheet.org/downloads/).



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob, yaml

from utils import get_match_list

data_dir='../datasets/t20s_male/'

In [2]:
f ='237242.yaml'

fil=data_dir+f
data=yaml.load(open(fil))

<ipython-input-2-4bb9cdb4bf93>:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data=yaml.load(open(fil))


In [3]:
print ( data.keys() )
print ()
print ( data['meta'] )
print ()
data['info']

dict_keys(['meta', 'info', 'innings'])

{'data_version': 0.9, 'created': datetime.date(2013, 2, 20), 'revision': 2}



{'bowl_out': [{'bowler': 'NJ Astle', 'outcome': 'miss'},
  {'bowler': 'NJ Astle', 'outcome': 'miss'},
  {'bowler': 'DR Smith', 'outcome': 'miss'},
  {'bowler': 'DR Smith', 'outcome': 'miss'},
  {'bowler': 'JS Patel', 'outcome': 'miss'},
  {'bowler': 'JS Patel', 'outcome': 'miss'},
  {'bowler': 'CH Gayle', 'outcome': 'miss'},
  {'bowler': 'CH Gayle', 'outcome': 'miss'},
  {'bowler': 'CL Cairns', 'outcome': 'miss'},
  {'bowler': 'CL Cairns', 'outcome': 'miss'},
  {'bowler': 'JE Taylor', 'outcome': 'miss'},
  {'bowler': 'JE Taylor', 'outcome': 'miss'},
  {'bowler': 'SE Bond', 'outcome': 'hit'},
  {'bowler': 'SE Bond', 'outcome': 'hit'},
  {'bowler': 'IDR Bradshaw', 'outcome': 'miss'},
  {'bowler': 'IDR Bradshaw', 'outcome': 'miss'},
  {'bowler': 'SB Styris', 'outcome': 'hit'}],
 'city': 'Auckland',
 'dates': [datetime.date(2006, 2, 16)],
 'gender': 'male',
 'match_type': 'T20',
 'outcome': {'bowl_out': 'New Zealand', 'result': 'tie'},
 'overs': 20,
 'player_of_match': ['DR Smith'],
 'team

In [4]:

def read_match_info(fil):
    with open(fil, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    try:
        match_date=data['info']['dates'][0].strftime('%Y-%m-%d')
    except:
        match_date=data['info']['dates'][0]

    season = match_date.split('-')[0]
    teams  = data['info']['teams']
    toss_winner   = data['info']['toss'].get('winner', None)
    toss_decision = data['info']['toss'].get('decision', None)
    batting_1st_team = list(data['innings'][0].items())[0][1]['team']
    player_of_match  = data['info'].get('player_of_match', [None])[0]

    print ('------------------------------------------------')
    print (teams[0] ,' vs ', teams[1], match_date)
    print ( 'Toss \t\t\t', toss_winner, ', Decided to', toss_decision)
    print ('Team Batting First\t', batting_1st_team)
    print ('Player of the Match\t', player_of_match )

    # take care of the tied match first
    if data['info']['outcome'].get('result', None)== 'tie':
        winner = data['info']['outcome'].get('bowl_out', None)
        print ('Result \t\t\t', 'Match tied.', winner, 'won the one over bowl out.')

    elif data['info']['outcome'].get('result', None)== 'no result':
        print ('No Results.')

    else:
        winner = data['info']['outcome'].get('winner', None)
        win_margin    = list(data['info']['outcome']['by'].values())[0]
        win_type      = list(data['info']['outcome']['by'].keys())[0]
        print ('Result \t\t\t', winner, 'won by ', win_margin, win_type )

    print ('------------------------------------------------')

    
fil1=data_dir+'1208609.yaml'
fil2=data_dir+'406198.yaml'

read_match_info(fil1)
print ()
read_match_info(fil2)

------------------------------------------------
Nepal  vs  Maldives 2019-12-06
Toss 			 Nepal , Decided to bat
Team Batting First	 Nepal
Player of the Match	 Karan KC
Result 			 Nepal won by  84 runs
------------------------------------------------

------------------------------------------------
Australia  vs  West Indies 2010-02-23
Toss 			 West Indies , Decided to bat
Team Batting First	 West Indies
Player of the Match	 DA Warner
Result 			 Australia won by  8 wickets
------------------------------------------------


In [10]:
%%writefile 'print_scorecard.py'
#!/usr/bin/env python3

import numpy as np
import yaml, os, sys
import pandas as pd

data_dir='../datasets/t20s_male/'

def print_scorecard(f, data_dir=data_dir):
    fil=os.path.join(data_dir, f )

    with open(fil, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    try:
        match_date=data['info']['dates'][0].strftime('%Y-%m-%d')
    except:
        match_date=data['info']['dates'][0]

    season = match_date.split('-')[0]
    teams  = data['info']['teams']
    winner = data['info']['outcome'].get('winner', None)
    toss_winner = data['info']['toss'].get('winner', None)

    tied = False
    if data['info']['outcome'].get('result', None)== 'tie':
        winner = data['info']['outcome'].get('bowl_out', None)
        tied = True

    def convert_to_ov(balls):
        ov, balls_ = (balls//6, balls%6)
        if balls_==0:
            return str(ov)
        else:
            return str(ov)+'.'+str(balls_)


    def add_batsman(player, batting_scorecard, season=season, Team=' ', Against=' '):
        if player not in batting_scorecard:
            batting_scorecard[player] = {'0s':0, '1s':0, '2s':0, '3s':0, '4s':0, '5s':0, '6s':0, '7s':0,
                                         'R':0, 'BF':0, 'NO':True, 'Team':Team, 'Against':Against, 'Tied':tied,
                                         'Win':False, 'Toss':False, 'team-total':0, 'season':season}

    def add_bowler(player,   bowling_scorecard, season=season, Team=' ', Against=' '):
        if player not in bowling_scorecard:
            bowling_scorecard[player] = {'O':0, 'M':0, 'R':0, 'W':0, 'ovs':{},
                    '0s':0, '1s':0, '2s':0, '3s':0, '4s':0, '5s':0, '6s':0, '7s':0,
                                         'WD':0, 'NB':0, 'Team':Team, 'Against':Against, 'Win':False, 'Tied':tied,
                                         'Toss':False, 'team-total':0, 'season':season}

    batting_card = {}
    bowling_card = {}

    innings = [list(d.keys())[0] for d in data['innings'] if 'Super' not in list(d.keys())[0] ]

    for i, inn_name in enumerate(innings):
        batting_team = data['innings'][i][inn_name]['team']
        bowling_team = [team for team in  teams if team!=batting_team][0]

        batting_card_inn={}
        bowling_card_inn={}

        runs_extra = 0
        runs_total = 0
        wkts       = 0

        for delivery in data['innings'][i][inn_name]['deliveries']:
            deliv    = list(delivery.items())[0]
            ball     = deliv[0]
            ov_num   = int(str(ball+1).split('.')[0])

            batsman  = deliv[1]['batsman'].strip()
            bowler   = deliv[1]['bowler'].strip()
            runs_bat = deliv[1]['runs'].get('batsman', 0)
            runs_ext = deliv[1]['runs'].get('extras',  0)
            runs_tot = deliv[1]['runs'].get('total',   0)

            add_batsman(batsman, batting_card_inn)
            add_bowler(bowler , bowling_card_inn)

            # counter for each runs (1, 2, 3, 4, 5, 6)
            batting_card_inn[batsman][str(runs_bat)+'s'] += 1
            bowling_card_inn[bowler ][str(runs_bat)+'s'] += 1

            # first check if the over exists on the dictionary
            if not bowling_card_inn[bowler ]['ovs'].get(ov_num, False):
                bowling_card_inn[bowler ]['ovs'][ov_num] = {'R':0, 'W':0}
            bowling_card_inn[bowler ]['ovs'][ov_num]['R'] += runs_tot

            # counter for total batsman run
            batting_card_inn[batsman]['R']           += runs_bat
            batting_card_inn[batsman]['BF']          += 1

            bowling_card_inn[bowler ]['R']           += runs_tot
            bowling_card_inn[bowler ]['O']           += 1

            if 'extras' in deliv[1]:
                if 'wides' in deliv[1]['extras']:
                    batting_card_inn[batsman]['BF'] -= 1 # remove the ball from batsman's account
                    bowling_card_inn[bowler]['O']   -= 1 # to count the extra ball
                    bowling_card_inn[bowler]['WD']  += 1 #

                elif 'noballs' in deliv[1]['extras']:
                    bowling_card_inn[bowler]['O']   -= 1 # to count the extra ball
                    bowling_card_inn[bowler]['NB']  += 1 #

                elif 'legbyes' in deliv[1]['extras']:
                    #bowling_card_inn[bowler]['R']   -= 1 #
                    bowling_card_inn[bowler]['R']   -= deliv[1]['extras']['legbyes']
                    bowling_card_inn[bowler ]['ovs'][ov_num]['R'] -= deliv[1]['extras']['legbyes']

                elif 'byes' in deliv[1]['extras']:
                    bowling_card_inn[bowler]['R']   -= deliv[1]['extras']['byes']

            runs_extra += runs_ext
            runs_total += runs_tot

            if deliv[1].get('wicket', None):
                wkts += 1
                player_out = deliv[1]['wicket']['player_out']

                # for case when player is runout without facing a ball
                add_batsman(player_out, batting_card_inn, Team=batting_team, Against=bowling_team)
                batting_card_inn[player_out]['NO'] = False

                # add wicket to bowler only if it's not RUN OUT
                if deliv[1]['wicket']['kind'] != 'run out':
                    bowling_card_inn[bowler ]['W']                += 1
                    bowling_card_inn[bowler ]['ovs'][ov_num]['W'] += 1

            batting_card_inn[batsman]['Team']    = batting_team
            batting_card_inn[batsman]['Against'] = bowling_team

            bowling_card_inn[bowler ]['Team']    = bowling_team
            bowling_card_inn[bowler ]['Against'] = batting_team

            if batting_team == winner:
                batting_card_inn[batsman]['Win'] = True
            else:
                bowling_card_inn[bowler ]['Win'] = True

            if batting_team == toss_winner:
                batting_card_inn[batsman]['Toss'] = True
            else:
                bowling_card_inn[bowler]['Toss'] = True

        for b in batting_card_inn.keys():
            batting_card_inn[b]['team-total'] = runs_total

        for b in bowling_card_inn.keys():
            bowling_card_inn[b]['team-total'] = runs_total
            bowling_card_inn[b]['O'] = convert_to_ov(bowling_card_inn[b]['O'])

        batting_card.update(batting_card_inn)
        bowling_card.update(bowling_card_inn)

    df_bat = pd.DataFrame(batting_card).transpose()

    df_bat.reset_index(inplace=True)
    df_bat.rename(columns={"index": "batsman"}, inplace=True)
    df_bat['date']=match_date
    df_bat['match-id']=f.split('/')[-1].split('.')[0]

    df_bowl = pd.DataFrame(bowling_card).transpose()

    df_bowl.reset_index(inplace=True)
    df_bowl.rename(columns={"index": "bowler"}, inplace=True)
    df_bowl['date']=match_date
    df_bowl['match-id']=f.split('/')[-1].split('.')[0]

    return (df_bat, df_bowl)

Overwriting print_scorecard.py


In [12]:
from print_scorecard import print_scorecard
import sys

def nice_scorecard(f, data_dir='./'):
    df_bat, df_bowl = print_scorecard(f, data_dir=data_dir)
    df_bat_nice  = df_bat[['batsman', 'R', 'BF', '4s', '6s', 'team-total']]
    df_bowl_nice = df_bowl[['bowler', 'O', 'M', 'R', 'W', 'WD', 'NB']]
    print (df_bat_nice)
    print (df_bowl_nice)

if __name__=="__main__":

    fil = data_dir + '237242.yaml'
    read_match_info(fil)
    nice_scorecard(fil)
    if 'full' in sys.argv:
        df_bat, df_bowl=print_scorecard(fil)
        print (df_bat, df_bowl)

------------------------------------------------
New Zealand  vs  West Indies 2006-02-16
Toss 			 New Zealand , Decided to field
Team Batting First	 West Indies
Player of the Match	 DR Smith
Result 			 Match tied. New Zealand won the one over bowl out.
------------------------------------------------
          batsman   R  BF 4s 6s team-total
0        CH Gayle  10  12  2  0        126
1         D Ganga  26  31  3  1        126
2       RS Morton   4   9  0  0        126
3        DR Smith   8   4  1  0        126
4        WW Hinds  14  18  1  0        126
5   S Chanderpaul  26  27  2  0        126
6        DJ Bravo  19  16  2  0        126
7        D Ramdin   9   4  2  0        126
8       L Vincent  42  37  5  1        126
9      SP Fleming   6   7  1  0        126
10       NJ Astle   5   7  1  0        126
11      SB Styris  19  16  2  1        126
12      CL Cairns   2   9  0  0        126
13      PG Fulton  16  17  0  0        126
14   HJH Marshall   1   4  0  0        126
15    BB M

In [13]:
df_bat

NameError: name 'df_bat' is not defined

In [ ]:
%%writefile

%writefile [-a] filename